In [ ]:
import gspread
import pandas as pd
from google.oauth2.service_account import Credentials

# 1. Define the scope
SCOPES = [
    "https://www.googleapis.com/auth/spreadsheets"
]

# 2. Authenticate using your Service Account key file
# Replace with the actual path to your JSON key file
SERVICE_ACCOUNT_FILE = '/Users/deepshah/Downloads/tiffinstash-key.json' 

creds = Credentials.from_service_account_file(SERVICE_ACCOUNT_FILE, scopes=SCOPES)
client = gspread.authorize(creds)

# 3. Open the Google Sheet
# You can open by title, url, or key. Opening by key is most robust.
# Example URL: https://docs.google.com/spreadsheets/d/1BxiMVs0XRA5nFMdKvBdBZjgmUUqptlbs74OgvE2upms/edit
SHEET_ID = '1yN7oUVMW_5R-JzEVwGQl26xnvUFay3aGWoudG5aMH48' 

try:
    # Open the spreadsheet
    sh = client.open_by_key(SHEET_ID)
    
    # Select the specific worksheet (tab) by index (0 is the first one) or name
    worksheet = sh.get_worksheet(3) 
    # OR: worksheet = sh.worksheet("Sheet1")

    # 4. Get all values and convert to DataFrame
    data = worksheet.get_all_records() # Returns a list of dictionaries
    df = pd.DataFrame(data)

    print("Data extracted successfully:")
    print(df.head())

except Exception as e:
    print(f"Error reading Google Sheet: {e}")



In [ ]:
import pandas as pd
import sqlalchemy
from sqlalchemy import text
from google.cloud.sql.connector import Connector, IPTypes
import os
from google.oauth2 import service_account



# # 1. Load your local CSV data
# file_path = os.path.expanduser('~/Downloads/historical-data-filtered.csv')
# df = pd.read_csv(file_path)

# 2. Cleanup dates as you did before
for col in df.columns:
    if df[col].dtype == 'object':
        try:
            df[col] = pd.to_datetime(df[col], dayfirst=True, format='mixed')
        except (ValueError, TypeError):
            continue

KEY_PATH = "/Users/deepshah/Downloads/tiffinstash-key.json"

credentials = service_account.Credentials.from_service_account_file(
    KEY_PATH
)

connector = Connector(credentials=credentials)

def getconn():
    conn = connector.connect(
        "pelagic-campus-484800-b3:us-central1:tiffinstash-master",
        "pg8000",
        user="postgres",
        password="tiffinstash2026",
        db="postgres",
        ip_type=IPTypes.PUBLIC
    )
    return conn

# 4. Create the SQLAlchemy Engine
engine = sqlalchemy.create_engine("postgresql+pg8000://", creator=getconn)

# 5. Import to the new table
# We use double quotes in the name because of the hyphen
table_name = 'historical-data'

try:
    # 'if_exists=replace' will create the table if it doesn't exist
    df.to_sql(table_name, engine, if_exists='replace', index=False)
    print(f"Successfully imported {len(df)} rows into '{table_name}'")
    
    # Verification query
    with engine.connect() as conn:
        res = conn.execute(text(f'SELECT COUNT(*) FROM "{table_name}"'))
        print(f"Verified row count in DB: {res.fetchone()[0]}")

except Exception as e:
    print(f"Import failed: {e}")

# Always clean up the connector
# connector.close()

In [3]:
import pandas as pd
import sqlalchemy
from sqlalchemy import text
from google.cloud.sql.connector import Connector, IPTypes
import os
from google.oauth2 import service_account


KEY_PATH = "/Users/deepshah/Downloads/tiffinstash-key.json"

credentials = service_account.Credentials.from_service_account_file(
    KEY_PATH
)

connector = Connector(credentials=credentials)

def getconn():
    conn = connector.connect(
        "pelagic-campus-484800-b3:us-central1:tiffinstash-master",
        "pg8000",
        user="postgres",
        password="tiffinstash2026",
        db="postgres",
        ip_type=IPTypes.PUBLIC
    )
    return conn

# 4. Create the SQLAlchemy Engine
engine = sqlalchemy.create_engine("postgresql+pg8000://", creator=getconn)

# 5. Import to the new table
# We use double quotes in the name because of the hyphen
table_name = 'historical-data'

with engine.connect() as conn:
    df = pd.read_sql(text(f'SELECT * FROM "{table_name}"'), conn)
    print(f"Verified row count in DB: {df.shape[0]}")
pd.set_option('display.max_columns', None)
df

Verified row count in DB: 5293


,ORDER ID,DATE,NAME,PHONE,EMAIL,HOUSE UNIT NO,ADDRESS LINE 1,CITY,ZIP,SKU,SELLER,DELIVERY,MEAL TYPE,MEAL PLAN,PRODUCT,PRODUCT CODE,CLABL,LABEL,DRIVER NOTE,SELLER NOTE,UPSTAIR DELIVERY,DELIVERY TIME,QUANTITY,DAYS,COUNT,START DATE,END DATE,STATUS,SKIP1,SKIP2,SKIP3,SKIP4,SKIP5,SKIP6,SKIP7,SKIP8,SKIP9,SKIP10,SKIP11,SKIP12,SKIP13,SKIP14,SKIP15,SKIP16,SKIP17,SKIP18,SKIP19,SKIP20,DELSAT,DELSUN,TS NOTES,DESCRIPTION
0,24451,2025-07-17 00:00:00,Mohit Gianani,4379253511,giananimohit@gmail.com,909,3939 Duke of York Blvd,Mississauga,L5B 4N2,INFLV-TD-MT26-M20-ONCA-INFNV,INFLV,TD,Deluxe Rice,Monthly,Indias Flavour NonVeg Mix,MT26(IF),0,MT26(IF),"Buzz code is 786. Call upon arrival, leave at ...",0,No,DINNER,1,2.0,2.0,"Mon, 12 Jan","Fri, 16 Jan",DELIVERED,13 Jan,14 Jan,15 Jan,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-,-,13 tiffins remaining,• 6 Roti • 1 Veg-NonVeg (16 oz) • 1 Curry (16 ...
1,24451,2025-07-17 00:00:00,Mohit Gianani,4379253511,giananimohit@gmail.com,909,3939 Duke of York Blvd,Mississauga,L5B 4N2,INFLV-TD-MT26-M20-ONCA-INFNV,INFLV,TD,Deluxe Rice,Monthly,Indias Flavour NonVeg Mix,MT26(IF),0,MT26(IF),"Buzz code is 786. Call upon arrival, leave at ...",0,No,DINNER,1,2.0,2.0,"Mon, 19 Jan","Fri, 23 Jan",DELIVERED,20 Jan,21 Jan,22 Jan,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-,-,-,• 6 Roti • 1 Veg-NonVeg (16 oz) • 1 Curry (16 ...
2,26401-WTD,2025-10-06 00:00:00,Dhruvi Vaidya,4379829557,dhruvi.vaidya@outlook.com,1613,330 McCowan Road,Scarborough,M1J 3N3,TPROS-TD-MT91-T01-ONCA-TPROS,TPROS,TD,Basic 2,Trial,Tiffin Pros,MT91(TP),0,MT91(TP),Sterling club ii. 1374- buzzer code. 16th floo...,0,Yes,DINNER,1,1.0,1.0,"Tue, 7 Oct","Tue, 7 Oct",DELIVERED,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-,-,-,• 4 Roti • 1 Veg (08 oz)
3,26401-WTD,2025-10-06 00:00:00,Dhruvi Vaidya,4379829557,dhruvi.vaidya@outlook.com,1613,330 McCowan Road,Scarborough,M1J 3N3,FIERY-TD-MT01-T01-ONCA-FGBVG,FIERY,TD,Basic,Trial,Fiery Grillss Budget Veg,MT01(FG),0,MT01(FG),Sterling club ii. 1374- buzzer code. 16th floo...,0,Yes,DINNER,1,1.0,1.0,"Wed, 8 Oct","Wed, 8 Oct",DELIVERED,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-,-,-,• 4 Roti • 1 Veg (08 oz)
4,26401-WTD,2025-10-06 00:00:00,Dhruvi Vaidya,4379829557,dhruvi.vaidya@outlook.com,1613,330 McCowan Road,Scarborough,M1J 3N3,LALKT-TD-MT31-T01-ONCA-SWAGT,LALKT,TD,Basic,Trial,Lal Budget Veg,MT31(LK),0,MT31(LK),Sterling club ii. 1374- buzzer code. 16th floo...,0,Yes,DINNER,1,1.0,1.0,"Thu, 9 Oct","Thu, 9 Oct",DELIVERED,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-,-,-,• 4 Roti • 1 Veg (08 oz)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5288,OD242,2026-01-31 00:00:00,CHAITNYA,(289) 834-4505,ritirivajtiffin@gmail.com,None,4023 THE EXCHANGE,Mississauga,L5B 0N4,RITAJ-TS-VD,RITAJ,TD,0,0,Seller Delivery,None,0,SD48(RV),please leave at reception area,nan,No,DINNER,1,NaN,NaN,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
5289,OD256,2026-01-31 00:00:00,Abhay Singh,2896750654,mummystiffincanada@gmail.com,146,146 Dowling ave,TORONTO,m6k3a6,MUMKT-TS-VD,MUMKT,TD,0,0,Seller Delivery,None,0,SD15(MU),buzz code 1607 and keep it at the proper place,nan,no,DINNER,1,NaN,NaN,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
5290,28815,2025-12-29 00:00:00,Suman Kotadia,4376082102,sumankotadia21@gmail.com,823,10 capreol court,Toronto,M5V 4B3,TSWAD-TD-MT02-M20-ONCA-FRESH,TSWAD,TD,Basic Kathod,Monthly,Swad Gujarati,MT02(SW),Suman Kot,Suman Kot,Leave at front desk and message me,"No eggplant, no cluster beans, and no okra. Pl...",No,DINNER,1,20.0,20.0,"Tue, 30 Dec","Mon, 2 Feb",WIP,31 Dec,1 Jan,2 Jan,15 Jan,26 Jan,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-,-,-,• 4 Roti • 1 Veg (08 oz) • 1 Kathod (08 Oz) • ...
5291,29788-WTD,2026-01-30 02:21:11,Rajdeep Singh

In [ ]:
a=123